## SPTrans: localizando ônibus que passam entre duas paradas
A SPtrans disponibiliza na sua [área para desenvolvedores](https://www.sptrans.com.br/desenvolvedores/) seus arquivos [GTFS](https://developers.google.com/transit/gtfs?hl=pt-br), um formato de informações de transporte público desenvolvido pelo Google. Estes arquivos são atualizados frequentemente. Os arquivos usados aqui foram baixados no final de janeiro de 2022.    
Algumas das informações disponibilizadas são:

- `trips.txt` linhas disponibilizadas
- `stops.txt` pontos de ônibus com localização geográfica
- `stop_times.txt` tempo estimado de viagem entre os pontos de cada linha

O formato **GTFS** é uma sugestão para as operadoras, que não necessariamente seguem à risca as suas especificações. O que descrevo abaixo são os arquivos e campos conforme utilizados pela SPTrans no momento deste texto (janeiro de 2022).

Com estas informações é possível levantar, por exemplo, quais linhas passam entre duas paradas específicas.

In [1]:
import pandas as pd
import numpy as np

O formato **GTFS** é equivalente a um arquivo *csv* com vírgula como separador.

In [2]:
stops_df = pd.read_csv('stops.txt', sep=',')
linhas = pd.read_csv('trips.txt', sep=',')
stop_times_df = pd.read_csv('stop_times.txt', sep=',')

In [3]:
stops_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20929 entries, 0 to 20928
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   stop_id    20929 non-null  int64  
 1   stop_name  20929 non-null  object 
 2   stop_desc  19584 non-null  object 
 3   stop_lat   20929 non-null  float64
 4   stop_lon   20929 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 817.7+ KB


#### Campos do arquivo `stops.txt`
- `stop_id` código númerico de identificação do ponto de ônibus
- `stop_name` nome do ponto de ônibus
- `stop_desc` informações adicionais sobre o ponto (opcional)
- `stop_lat` Latitude do local (em formato decimal)
- `stop_lon` Longitude do local (em formato decimal)

In [4]:
linhas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2223 entries, 0 to 2222
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   route_id       2223 non-null   object
 1   service_id     2223 non-null   object
 2   trip_id        2223 non-null   object
 3   trip_headsign  2223 non-null   object
 4   direction_id   2223 non-null   int64 
 5   shape_id       2223 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 104.3+ KB


#### Campos do arquivo `trips.txt`
- `route_id` código de identificação do trajeto
- `service_id` código que identifica os dias da semana que a linha funciona
- `trip_id` código da linha de ônibus
- `trip_headsign` letreiro do ônibus durante o trajeto
- `direction_id` direção da viagem (ida/volta)
- `shape_id` Identifica uma forma geoespacial que descreve o caminho do veículo em uma viagem (da especificação, provavelmente associado ao arquivo `shapes.txt`)

In [5]:
stop_times_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95382 entries, 0 to 95381
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   trip_id         95382 non-null  object
 1   arrival_time    95382 non-null  object
 2   departure_time  95382 non-null  object
 3   stop_id         95382 non-null  int64 
 4   stop_sequence   95382 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 3.6+ MB


#### Campos do arquivo `stop_times.txt`
- `trip_id` código da linha de ônibus
- `arrival_time` horário de chegada ao ponto
- `departure_time` horário de saída do ponto
- `stop_id` código númerico de identificação do ponto de ônibus
- `stop_sequence` numeração da parada na sequência de paradas referentes à linha de ônibus

Algumas observações sobre este arquivo:

In [6]:
stop_times_df.loc[stop_times_df['trip_id'] == '372F-10-0']

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
26699,372F-10-0,07:00:00,07:00:00,540014162,1
26700,372F-10-0,07:02:34,07:02:34,540014163,2
26701,372F-10-0,07:05:08,07:05:08,540014457,3
26702,372F-10-0,07:07:42,07:07:42,5410182,4
26703,372F-10-0,07:10:16,07:10:16,5410183,5
26704,372F-10-0,07:12:50,07:12:50,540014258,6
26705,372F-10-0,07:18:00,07:18:00,540014257,7


*Curiosidade: a linha acima é identificada como a mais curta de S. Paulo e só tem estas 7 paradas.*

A especificação do [GTFS](https://developers.google.com/transit/gtfs/reference?hl=pt-br#stop_timestxt) não ficou clara para mim; neste arquivo da SPTrans, como visto no exemplo acima, aparentemente informa-se os tempos de apenas uma viagem.  
Note-se que no exemplo `arrival_time` e `departure_time` são os mesmos.

### Como localizar uma parada
Teoricamente todas as paradas de ônibus em S. Paulo tem um cartaz de identificação como [este](https://www.sptrans.com.br/media/3714/comunicacao2019.jpg), informando o nome do ponto.  
Aqui vou tentar achar pelo nome duas paradas que já identifiquei, mas é possível também usar as informações de latitude e longitude (por exemplo, informadas pelo GPS de um *smartphone*).

[Parada Paulista B/C](https://goo.gl/maps/zhAXKtPutGxWkuL59)

In [9]:
paulista = stops_df.loc[stops_df['stop_name'] == 'Paulista B/C', 'stop_id'].values[0]

[Parada São Luis](https://goo.gl/maps/ZjBNCyCcrS6zasyZ9)

In [10]:
sao_luis = stops_df.loc[stops_df['stop_name'] == 'São Luis', 'stop_id'].values[0]

Aqui eu seleciono as linhas de ônibus que passam em um ou outro ponto (ou seja, passam nos dois).

In [11]:
onibus1 = stop_times_df.loc[(stop_times_df['stop_id'] == paulista)]
onibus1

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
7442,178L-10-1,17:12:12,17:12:12,260015039,7
64569,701A-10-1,15:29:06,15:29:06,260015039,19
64733,701U-10-1,16:59:24,16:59:24,260015039,34
64947,702C-10-0,17:04:30,17:04:30,260015039,31
65052,702U-10-0,16:55:21,16:55:21,260015039,28
66733,7267-10-0,16:23:50,16:23:50,260015039,12
66786,7272-10-0,16:47:36,16:47:36,260015039,29
66845,7281-10-0,15:36:06,15:36:06,260015039,20
66934,7282-10-0,16:19:03,16:19:03,260015039,52
67208,7411-10-0,16:37:12,16:37:12,260015039,19


In [12]:
onibus2 = stop_times_df.loc[(stop_times_df['stop_id'] == sao_luis)]
onibus2

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
65056,702U-10-0,17:03:33,17:03:33,670016882,32
66790,7272-10-0,16:54:24,16:54:24,670016882,33
67212,7411-10-0,16:45:28,16:45:28,670016882,23
67280,7458-10-0,08:15:50,08:15:50,670016882,36
68261,7545-10-0,08:13:07,08:13:07,670016882,42
68339,7545-21-0,09:14:00,09:14:00,670016882,33
69621,7903-10-0,17:14:04,17:14:04,670016882,45
78113,8700-1-0,08:10:24,08:10:24,670016882,34
78185,8700-10-0,08:10:50,08:10:50,670016882,35
78312,8705-10-0,17:02:20,17:02:20,670016882,35


In [13]:
resultado = np.intersect1d(onibus1['trip_id'].unique(), onibus2['trip_id'].unique())
resultado_df = pd.DataFrame(resultado, columns=['linhas'])

Buscando os letreiros dos ônibus.

In [14]:
linhas[linhas['trip_id'].isin(resultado_df['linhas'])]

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
1513,702U-10,USD,702U-10-0,Term. Pq. D. Pedro Ii,0,57025
1573,7272-10,US_,7272-10-0,Pça. Ramos De Azevedo,0,78578
1583,7411-10,U__,7411-10-0,Pça. Da Sé,0,54211
1584,7458-10,USD,7458-10-0,Est. Da Luz,0,71953
1604,7545-10,USD,7545-10-0,Pça. Ramos De Azevedo,0,71827
1606,7545-21,U__,7545-21-0,Pça. Ramos De Azevedo,0,54573
1629,7903-10,USD,7903-10-0,Pça. Ramos De Azevedo,0,56541
1828,8700-1,USD,8700-1-0,Pça. Ramos De Azevedo,0,64216
1830,8700-10,USD,8700-10-0,Pça. Ramos De Azevedo,0,63593
1834,8705-10,US_,8705-10-0,Anhangabaú,0,73016


Explicando melhor o resultado acima:  
- `route_id` é o código do ônibus
- `service_id` são os dias da semana que a linha funciona:
    - `U` dias úteis
    - `S` sábado
    - `D` domingo
- `direction_id` é o sentido da linha, `0` ida e `1` volta